In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import jax
import pandas as pd
import numpy as np
import jax.numpy as jnp
from utilities.adding_noise import adding_noise_to_observations
from jax import random, vmap, jit, grad, lax
import jax
from jax import random
from jax.nn.initializers import normal, zeros
from jax.example_libraries import stax
from jax.example_libraries import optimizers

import pickle
import glob

import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.tri as tri
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm  # Colour map
import matplotlib.animation as animatio
import matplotlib.font_manager
mpl.rcParams['figure.figsize'] = [5, 5/1.3333]
mpl.rcParams['figure.dpi'] = 300
params = {'legend.fontsize': 9, 'axes.labelsize': 9, 'xtick.labelsize':9, 'ytick.labelsize':9}
pylab.rcParams.update(params)
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'

In [2]:
num_test = 500
case = 8
key_noise_train = random.PRNGKey(10 + case)
key_noise_test = random.PRNGKey(20 + case)
key_network = 100
noise_level = 0.005

n = 15
num_observation = 10  # number of observed points
dimension_of_PoI = (n + 1)**2  # external force field
num_truncated_series = 15

neurons = 5000


## Loading test data


In [3]:

def load_data(name, target_shape=(-1,)):
    return np.reshape(pd.read_csv(name).to_numpy(), target_shape)
train_Observations_synthetic = load_data('data/poisson_2D_state_obs_train_o10_d10000_n15_AC_1_1_pt5.csv', (10000, -1))
train_Parameters = load_data('data/poisson_2D_parameter_train_d10000_n15_AC_1_1_pt5.csv', (10000, -1))

#? 1.1 Add noise (WE DO NOT USE vmap, BE CAREFUL! )

test_Observations_synthetic = load_data('data/poisson_2D_state_obs_test_o10_d' + str(num_test) + '_n15_AC_1_1_pt5.csv', (num_test, -1))
test_Parameters = load_data('data/poisson_2D_parameter_test_d' + str(num_test) + '_n15_AC_1_1_pt5.csv', (num_test, -1))

#? 1.1 Add noise (WE DO NOT USE vmap, BE CAREFUL! )
train_Observations = adding_noise_to_observations(train_Observations_synthetic, noise_level, key_noise_train)
test_Observations = adding_noise_to_observations(test_Observations_synthetic, noise_level, key_noise_test)

full_true_obs_train =np.reshape(pd.read_csv('data/poisson_2D_state_full_train_d10000_n15_AC_1_1_pt5.csv').to_numpy(), (10000, -1))[0,:]
full_true_obs =np.reshape(pd.read_csv('data/poisson_2D_state_full_test_d500_n15_AC_1_1_pt5.csv').to_numpy(), (500, -1))

#? 1.2 Load Eigenvalue, Eigenvectors, observed indices, prematrices
Eigen = load_data('data/prior_mean_n15_AC_1_1_pt5.csv', (dimension_of_PoI, num_truncated_series))
Sigma = load_data('data/prior_covariance_n15_AC_1_1_pt5.csv', (num_truncated_series, num_truncated_series))
obs_indices = load_data('data/poisson_2D_obs_indices_o10_n15.csv', (num_observation, -1))
free_index = load_data('data/prior_covariance_cholesky_n15_AC_1_1_pt5.csv')

Tik_POI = np.loadtxt('data/Tik_solutions.txt', delimiter = ',')

## Encoder decoder neural networks

In [4]:
def neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network):
    #? 2.0 Define Obs_2_PoI_NN and PoI_2_Obs_NN architectures
    Obs_2_PoI_NN_init, Obs_2_PoI_NN = stax.serial(
        stax.Dense(neurons, W_init=normal(0.02), b_init=zeros), stax.Relu,
        stax.Dense(num_truncated_series, W_init=normal(0.02), b_init=zeros)
    )

    PoI_2_Obs_NN_init, PoI_2_Obs_NN = stax.serial(
        stax.Dense(neurons, W_init=normal(0.02), b_init=zeros), stax.Relu,
        stax.Dense(210, W_init=normal(0.02), b_init=zeros)
    )
    
    #? 2.1 Randomize the weights and bias
    enc_init_rng, dec_init_rng = random.split(random.PRNGKey(key_network))
    
    _, init_Obs_2_PoI_NN_params = Obs_2_PoI_NN_init(enc_init_rng, (num_observation,))
    _, init_PoI_2_Obs_NN_params = PoI_2_Obs_NN_init(dec_init_rng, (num_truncated_series,))

    init_params = init_Obs_2_PoI_NN_params, init_PoI_2_Obs_NN_params
    return Obs_2_PoI_NN, PoI_2_Obs_NN, init_params

def neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network):
    #? 2.0 Define Obs_2_PoI_NN and PoI_2_Obs_NN architectures
    Obs_2_PoI_NN_init, Obs_2_PoI_NN = stax.serial(
        stax.Dense(neurons, W_init=normal(0.02), b_init=zeros), stax.Relu,
        stax.Dense(num_truncated_series, W_init=normal(0.02), b_init=zeros)
    )

    PoI_2_Obs_NN_init, PoI_2_Obs_NN = stax.serial(
        stax.Dense(neurons, W_init=normal(0.02), b_init=zeros), stax.Relu,
        stax.Dense(num_observation, W_init=normal(0.02), b_init=zeros)
    )
    
    #? 2.1 Randomize the weights and bias
    enc_init_rng, dec_init_rng = random.split(random.PRNGKey(key_network))

    _, init_Obs_2_PoI_NN_params = Obs_2_PoI_NN_init(enc_init_rng, (num_observation,))
    _, init_PoI_2_Obs_NN_params = PoI_2_Obs_NN_init(dec_init_rng, (num_truncated_series,))

    init_params = init_Obs_2_PoI_NN_params, init_PoI_2_Obs_NN_params
    return Obs_2_PoI_NN, PoI_2_Obs_NN, init_params



## Accuracy functions

In [5]:
def solution_to_observation(free_index, obs_indices, num_observation):
    i = 0
    obs_transfered = []
    for free_ind in free_index:
        if (free_ind in obs_indices):
            obs_transfered.append(i)
        i += 1

    jjj = 0
    obs_operator = np.zeros((num_observation, free_index.shape[0]))
    for obs_index in obs_transfered:
        obs_operator[jjj, obs_index] = 1
        jjj += 1
        
    return jnp.asarray(obs_operator)

obs_operator = solution_to_observation(free_index, obs_indices, num_observation)
def Observation_operator(full_state):
    return jnp.dot(obs_operator, full_state)


batched_Observation_operator = jax.jit(vmap(Observation_operator))
    
# ! 4.2 Accuracy rate function
def acc_PoI(preds, true, Eigen, Sigma):
        return jnp.mean(jnp.square(Eigen @ Sigma @ (preds - true))) / jnp.mean(jnp.square(Eigen @ Sigma @ true))

def acc_obs(preds, true):
    return jnp.mean(jnp.square(preds - true)) / jnp.mean(jnp.square(true))

batched_acc_PoI = vmap(acc_PoI, in_axes=(0, 0, None, None))
batched_acc_obs = vmap(acc_obs, in_axes=(0, 0))

@jit
def accuracy_data_P2_full_O(params, Observations, PoI, Eigen, Sigma):
    init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained = params

    Obs_2_PoI_NN_preds = Obs_2_PoI_NN(init_Obs_2_PoI_NN_params_trained, Observations)
    Obs_2_PoI_NN_Err_all = batched_acc_PoI(Obs_2_PoI_NN_preds, PoI, Eigen, Sigma)
    
    PoI_2_Obs_NN_preds = batched_Observation_operator(PoI_2_Obs_NN(init_PoI_2_Obs_NN_params_trained, PoI))
    PoI_2_Obs_NN_Err_all = batched_acc_obs(PoI_2_Obs_NN_preds, Observations)
    
    return Obs_2_PoI_NN_Err_all.mean(), PoI_2_Obs_NN_Err_all.mean(), Obs_2_PoI_NN_Err_all.std(), PoI_2_Obs_NN_Err_all.std()

@jit
def accuracy_data_P2_sparse_O(params, Observations, PoI, Eigen, Sigma):
    init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained = params

    Obs_2_PoI_NN_preds = Obs_2_PoI_NN(init_Obs_2_PoI_NN_params_trained, Observations)
    Obs_2_PoI_NN_Err_all = batched_acc_PoI(Obs_2_PoI_NN_preds, PoI, Eigen, Sigma)
    
    PoI_2_Obs_NN_preds = PoI_2_Obs_NN(init_PoI_2_Obs_NN_params_trained, PoI)
    PoI_2_Obs_NN_Err_all = batched_acc_obs(PoI_2_Obs_NN_preds, Observations)
    
    return Obs_2_PoI_NN_Err_all.mean(), PoI_2_Obs_NN_Err_all.mean(), Obs_2_PoI_NN_Err_all.std(), PoI_2_Obs_NN_Err_all.std()




## Generate the table

In [6]:
import numpy
print(numpy.__version__)

1.21.6


In [7]:

name_learning_approach_array = ['$\purePOP$', '$\pureOPO$', '$\mcPOP$', '$\mcOPO$', '$\mcOPOfull$', '$\TNetAE$', '$\TNetAEfull$']
for name_learning_approach in name_learning_approach_array:
    for num_train in [1, 100]:
        if name_learning_approach == '$\purePOP$':
            name_learning = 'Naive_AutoEncoder_P2O2P'
            best_inv = 'STEP_2'
            best_for = 'STEP_1'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            
        if (name_learning_approach == '$\mcPOP$'):
            name_learning = 'mc_AutoEncoder_P2O2P'
            best_inv = 'STEP_2'
            best_for = 'STEP_1'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            
        if name_learning_approach == '$\pureOPO$':
            name_learning = 'Naive_AutoEncoder_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            
        if (name_learning_approach == '$\mcOPO$'):
            name_learning = 'mc_AutoEncoder_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            
        if (name_learning_approach == '$\TNetAE$'):
            name_learning = 'mc_AutoEncoder_TNet_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            
        if (name_learning_approach == '$\TNetAEfull$'):
            name_learning = 'mc_AutoEncoder_TNet_O2P2O_modified'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_full_O
            
        if (name_learning_approach == '$\mcOPOfull$'):
            name_learning = 'mc_AutoEncoder_O2P2O_modified'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_full_O
            
            
        filename_pattern = f'Foward_{best_for}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network/Network/{filename_pattern}')
        
        # print(matching_files[0])
        
        best_for_params = pickle.load(open(matching_files[0], 'rb'))
        # opt_int, opt_update, opt_get_params = optimizers.adam(1e-3)
        # For_opt_state = optimizers.pack_optimizer_state(best_for_params)
        if best_for == 'STEP_1':
            _, init_PoI_2_Obs_NN_params_trained  = best_for_params
        if best_for == 'STEP_2':
            init_PoI_2_Obs_NN_params_trained  = best_for_params
        
        filename_pattern = f'Inverse_{best_inv}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network/Network/{filename_pattern}')
        best_inv_params = pickle.load(open(matching_files[0], 'rb'))
        # opt_int, opt_update, opt_get_params = optimizers.adam(1e-3)
        # For_opt_state = optimizers.pack_optimizer_state(best_inv_params)
        if best_inv == 'STEP_1':
            init_Obs_2_PoI_NN_params_trained, _  = best_inv_params
        if best_inv == 'STEP_2':
            init_Obs_2_PoI_NN_params_trained  = best_inv_params
        
        Obs_2_PoI_NN_accuracy_test, PoI_2_Obs_NN_accuracy_test, Obs_2_PoI_NN_accuracy_test_std, PoI_2_Obs_NN_accuracy_test_std = acc_function((init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained), test_Observations, test_Parameters, Eigen, Sigma)
        # print(f'{name_learning_approach} {num_train} {Obs_2_PoI_NN_accuracy_test * 100 : .2f}+-{Obs_2_PoI_NN_accuracy_test_std * 100 : .2f} and {PoI_2_Obs_NN_accuracy_test :.2e}+-{PoI_2_Obs_NN_accuracy_test_std :.2e}')
        print(f'{name_learning_approach} {num_train} {Obs_2_PoI_NN_accuracy_test * 100 : .2f} and {PoI_2_Obs_NN_accuracy_test :.2e}')
        
        

$\purePOP$ 1  100.18 and 3.99e-01
$\purePOP$ 100  80.48 and 5.30e-02
$\pureOPO$ 1  107.55 and 2.90e-01
$\pureOPO$ 100  50.18 and 1.09e-01
$\mcPOP$ 1  107.99 and 3.99e-01
$\mcPOP$ 100  87.60 and 5.30e-02
$\mcOPO$ 1  108.28 and 2.73e-02
$\mcOPO$ 100  46.32 and 3.94e-04
$\mcOPOfull$ 1  108.28 and 4.21e-02
$\mcOPOfull$ 100  46.32 and 1.56e-04
$\TNetAE$ 1  45.03 and 1.57e-04
$\TNetAE$ 100  44.73 and 1.22e-04
$\TNetAEfull$ 1  45.03 and 8.80e-04
$\TNetAEfull$ 100  44.73 and 2.12e-04


## Plot quantative comparison plots for inverse and forward observations for different approaches

### loading Fenics transformation and triangulation for plotting over the domain

In [8]:

reorder_index = np.load('data/reorder_vector_field.npy').astype(int)   

# Load the saved mesh data
with open('data/mesh_data.pickle', 'rb') as f:
    mesh_data = pickle.load(f)

coords = mesh_data['coords']
elements = mesh_data['elements']
obs_coords = mesh_data['obs_coords']

# Create triangulation
triangulation = tri.Triangulation(coords[:, 0], coords[:, 1], elements)


In [9]:
# Plot the triangulation
plt.figure(figsize=(4, 4))
plt.triplot(triangulation, 'k-', linewidth=0.5)
plt.plot([0,0], [0,1], 'b', linewidth=2.)
plt.plot([0,1], [0,0], 'b', linewidth=2.)
plt.plot([0,1], [1,1], 'b', linewidth=2., label = r'$\Gamma^{ext}$')
plt.plot([1,1], [0,1], 'r', linewidth=2., label = r'$\Gamma^{root}$')
plt.plot(obs_coords[:,0, 0], obs_coords[:,0, 1], 'ks', markersize=3, label = 'Observed points')
plt.axis('equal')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.legend(ncol = 3, bbox_to_anchor=(0.5, -0.07), loc='upper center')

# Hide the axis lines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
# plt.show()

import tikzplotlib
tikzplotlib.save(f"../../mcAE_Overleaf/Figs/2D_Poisson/domain_mesh_observables.tex")

plt.close()

In [10]:

def plot_domain(nodal_values, max, min, de, filename = '', save = True, cmap_plot = 'Reds', gap = 0.1, position = 'vertical', scale=1., observed_points = True):

    # Create triangulation
    triangulation = tri.Triangulation(coords[:, 0], coords[:, 1], elements)

    # Plot
    fig, ax = plt.subplots()
    ax.set_aspect('equal')

    # Refine the triangulation
    refiner = tri.UniformTriRefiner(triangulation)
    tri_refi, z_test_refi = refiner.refine_field(nodal_values, subdiv=3)

    # Plot contours
    levels = np.arange(min, max, de)
    cmap = plt.get_cmap(cmap_plot)
    tcf = ax.tricontourf(tri_refi, z_test_refi, levels=levels, cmap=cmap)
    cbar = plt.colorbar(tcf, shrink=scale, orientation=position, pad=0.05)
    cbar.set_ticks(np.arange(min, max, gap))

    if observed_points:
        # Plot observation points if available
        for point in obs_coords[:,0,:]:
            ax.plot(point[0], point[1], 'ks', mew=2, ms=2)


    ax.set_xticks([])
    ax.set_yticks([])

    if save:
        plt.savefig(filename, bbox_inches='tight')
        plt.close()
    else:
        plt.show()
        
    
def plot_single_true_versus_test(True_solutions, Pred_solutions, filename = ' ', savefig = False, lim_min = 100., lim_max = 100.):
    fig = plt.figure()
    fig.patch.set_facecolor('xkcd:white')
    
    if lim_min == 100.:
        lim_min = jnp.round(jnp.asarray([True_solutions.min(), Pred_solutions.min()]).min(),1) -.2
        lim_max = jnp.round(jnp.asarray([True_solutions.max(), Pred_solutions.max()]).max(),1) +.2
    
    plt.scatter(True_solutions, Pred_solutions)
    plt.gca().set_aspect('equal', adjustable='box')
    
    lim_min = 0.
    lim_max = 3.
    
    axis_range = np.linspace(lim_min, lim_max, 5)
    plt.xticks(axis_range), plt.yticks(axis_range)
    
    plt.xlim(lim_min,lim_max), 
    plt.ylim(lim_min,lim_max)
    x = np.linspace(lim_min, lim_max, 1000)
    plt.plot(x,x, 'r', linewidth=3)
    
    # current_values = plt.gca().get_xticks()
    # plt.gca().set_yticklabels(['{:,.1f}'.format(x) for x in current_values])
    # plt.gca().set_xticklabels(['{:,.1f}'.format(x) for x in current_values])
    plt.xticks([]), plt.yticks([])
    
    if savefig:
        # tikzplotlib.save(f"{filename}.tex")
        plt.savefig(f'{filename}.pdf', bbox_inches='tight')
        plt.close()
    else:
        plt.show()

    
# plot_domain(nodal_values, max, min, de)

In [12]:

data = [ ]

name_learning_approach_array = ['$\purePOP$', '$\pureOPO$', '$\mcPOP$', '$\mcOPO$', '$\mcOPOfull$', '$\TNetAE$', '$\TNetAEfull$']
# name_learning_approach_array = ['$\purePOP$']
for name_learning_approach in name_learning_approach_array:
    for num_train in [1, 100]:
        if name_learning_approach == '$\purePOP$':
            name_learning = 'Naive_AutoEncoder_P2O2P'
            best_inv = 'STEP_2'
            best_for = 'STEP_1'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
                
        if (name_learning_approach == '$\mcPOP$'):
            name_learning = 'mc_AutoEncoder_P2O2P'
            best_inv = 'STEP_2'
            best_for = 'STEP_1'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
                
        if name_learning_approach == '$\pureOPO$':
            name_learning = 'Naive_AutoEncoder_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
                
        if (name_learning_approach == '$\mcOPO$'):
            name_learning = 'mc_AutoEncoder_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
                
        if (name_learning_approach == '$\TNetAE$'):
            name_learning = 'mc_AutoEncoder_TNet_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
        if (name_learning_approach == '$\TNetAEfull$'):
            name_learning = 'mc_AutoEncoder_TNet_O2P2O_modified'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_full_O
            
        if (name_learning_approach == '$\mcOPOfull$'):
            name_learning = 'mc_AutoEncoder_O2P2O_modified'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_full_O
            
        filename_pattern = f'Foward_{best_for}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network/Network/{filename_pattern}')
        best_for_params = pickle.load(open(matching_files[0], 'rb'))
        if best_for == 'STEP_1':
            _, init_PoI_2_Obs_NN_params_trained  = best_for_params
        if best_for == 'STEP_2':
            init_PoI_2_Obs_NN_params_trained  = best_for_params
        
        filename_pattern = f'Inverse_{best_inv}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network/Network/{filename_pattern}')
        best_inv_params = pickle.load(open(matching_files[0], 'rb'))
        if best_inv == 'STEP_1':
            init_Obs_2_PoI_NN_params_trained, _  = best_inv_params
        if best_inv == 'STEP_2':
            init_Obs_2_PoI_NN_params_trained  = best_inv_params
        
        # Obs_2_PoI_NN_accuracy_test, PoI_2_Obs_NN_accuracy_test, _, _ = acc_function((init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained), test_Observations, test_Parameters, Eigen, Sigma)
        # print(f'{name_learning_approach} {num_train} {Obs_2_PoI_NN_accuracy_test * 100 : .2f} {PoI_2_Obs_NN_accuracy_test :.2e}')
        
        PoI_test_pred = PoI_2_Obs_NN(init_Obs_2_PoI_NN_params_trained, test_Observations)
        Obs_test_pred = Obs_2_PoI_NN(init_PoI_2_Obs_NN_params_trained, test_Parameters)
        
        diction = {
                        "name_learning_approach": name_learning_approach,
                        "num_train": num_train,
                        "PoI_test_pred": PoI_test_pred,
                        "Obs_test_pred": Obs_test_pred,
                    }
        
        data.append(diction)
        
        # # plot_single_true_versus_test(PoI_test_pred, test_Parameters) WE DO NOT PLOT THIS IN THE PAPER
        
        file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/forward_predictions_{name_learning}_d{num_train}'
        if (name_learning_approach == '$\TNetAEfull$') or (name_learning_approach == '$\mcOPOfull$'):
            plot_single_true_versus_test(batched_Observation_operator(Obs_test_pred), test_Observations, filename = file_name_to_save_fig, savefig = True)
        else:
            plot_single_true_versus_test(Obs_test_pred, test_Observations, filename = file_name_to_save_fig, savefig = True)

        
        @jax.vmap
        def from_z_to_kappa(z):
                return jnp.dot(z, jnp.transpose(jnp.dot(Eigen, Sigma)))

        # bb = (np.square(from_z_to_kappa(test_Parameters)).mean(axis = 0))
        # aa = np.square(from_z_to_kappa(PoI_test_pred) - from_z_to_kappa(test_Parameters))
        # AA = np.einsum('ij,j->ij', aa, 1/bb) * 100
        # nodal_values = np.asarray(AA).mean(axis = 0)
        # nodal_values_std = np.asarray(AA).std(axis = 0)
        
        nodal_values = np.asarray(np.abs(from_z_to_kappa(PoI_test_pred) - from_z_to_kappa(test_Parameters))).mean(axis = 0)
        nodal_values_std = np.asarray(np.abs(from_z_to_kappa(PoI_test_pred) - from_z_to_kappa(test_Parameters))).std(axis = 0)
        
        nodal_values = nodal_values[reorder_index]
        nodal_values_std = nodal_values_std[reorder_index]
        
        # print(nodal_values.max(), nodal_values.min())
        # print(nodal_values_std.max(), nodal_values_std.min())
        
        # file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/inverse_mean_error_predictions_{name_learning}_d{num_train}.pdf'
        # plot_domain(nodal_values, 0.41, 0., 0.02, filename = file_name_to_save_fig, save = True)
        
        file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/inverse_std_error_predictions_{name_learning}_d{num_train}.pdf'
        plot_domain(nodal_values_std, 0.31, 0., 0.02, filename = file_name_to_save_fig, save = True)
        
# df = pd.DataFrame(data).reset_index(drop=True)

In [13]:
nodal_values = np.asarray(np.abs(from_z_to_kappa(Tik_POI) - from_z_to_kappa(test_Parameters))).mean(axis = 0)
nodal_values_std = np.asarray(np.abs(from_z_to_kappa(Tik_POI) - from_z_to_kappa(test_Parameters))).std(axis = 0)
nodal_values = nodal_values[reorder_index]
nodal_values_std = nodal_values_std[reorder_index]

file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/inverse_mean_error_predictions_Tikhonov.pdf'
plot_domain(nodal_values, 0.41, 0., 0.02, filename = file_name_to_save_fig)

file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/inverse_std_error_predictions_Tikhonov.pdf'
plot_domain(nodal_values_std, 0.31, 0., 0.02, filename = file_name_to_save_fig)

In [ ]:
def output_to_numpy(df, name_learning_approach, num_train, inverse_or_forward):
    if inverse_or_forward == 'inv':
        A = df[df['num_train'] == num_train]
        output = A[A['name_learning_approach']==name_learning_approach]['PoI_test_pred'].to_numpy()[0]
    if inverse_or_forward == 'for':
        A = df[df['num_train'] == num_train]
        output = A[A['name_learning_approach']==name_learning_approach]['Obs_test_pred'].to_numpy()[0]
    return output

In [ ]:
# def plot_inv(name_learning_approach, num_train, color_line, dash):
#     output = output_to_numpy(df, '$' + name_learning_approach + '$', num_train, 'inv')
#     sns.histplot(x=output.flatten(), stat = 'probability', bins = 81, binrange = [-8,8], element="poly", color = color_line, dashes = dash, alpha = 1, fill=False, label = '$' + name_learning_approach )
    

# sns.histplot(x=test_Parameters.flatten(), stat = 'probability', bins = 81, binrange = [-8,8], element="poly", color = 'k', dashes = [2,2], alpha = 1, fill=False, legend='true', label = 'True')
# sns.histplot(x=Tik_POI.flatten(), stat = 'probability', bins = 81, binrange = [-8,8], element="poly", color = 'y', dashes = [2,2], alpha = 1, fill=False, legend='true', label = 'Tikhonov')
# plot_inv('\purePOP', 100, 'c', (1,1))
# plot_inv('\pureOPO', 100, 'k', (1,1))
# plot_inv('\mcPOP', 100, 'm', (1,1))
# plot_inv('\mcOPO', 100, 'g', (2,2))
# plot_inv('\TNetAE', 100, 'b', (2,2))
# plot_inv('\TNetAEfull', 100, 'r', (2,2))
# plt.legend()
# plt.xlim(-4,4)
# plt.show()
    

## plot the 1st training sample (converted triangulaization plot from Fenics)



In [ ]:
def from_z_to_kappa(z):
        return jnp.dot(z, jnp.transpose(jnp.dot(Eigen, Sigma)))
    
nodal_values = np.asarray(from_z_to_kappa(train_Parameters)[0,:])
nodal_values = nodal_values[reorder_index]
max, min, de = .9, -.8, 0.1

file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/1st_training_PoI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap_plot = 'jet', gap = .4, position = 'horizontal', scale = 0.6, observed_points = False)

In [ ]:
nodal_values = full_true_obs_train
nodal_values = nodal_values[reorder_index]

max, min, de = 3.1, 0., 0.1
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/1st_training_yobs_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, gap = 0.6, position = 'horizontal', scale = 0.6)

## predictions by TNet full

In [ ]:
def from_210_to_full(free_index_obs_pred):
    Operator = np.zeros((210, 256))
    i = 0
    for j in free_index:
        Operator[i, j] = 1
        i += 1
    return np.einsum('ij, bi -> bj', Operator, free_index_obs_pred)
    
name_learning_approach_array = ['$\mcOPOfull$', '$\TNetAEfull$']
for name_learning_approach in name_learning_approach_array:
    # for num_train in [1,50,100]:
    for num_train in [100,1]:
        if (name_learning_approach == '$\mcOPOfull$'):
            name = 'mc_full'
            name_learning = 'mc_AutoEncoder_O2P2O_modified'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_full_O
            
        if (name_learning_approach == '$\TNetAEfull$'):
            name = 'Tnet_full'
            name_learning = 'mc_AutoEncoder_TNet_O2P2O_modified'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_full_O
            
            
        filename_pattern = f'Foward_{best_for}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network/Network/{filename_pattern}')
        best_for_params = pickle.load(open(matching_files[0], 'rb'))
        if best_for == 'STEP_1':
            _, init_PoI_2_Obs_NN_params_trained  = best_for_params
        if best_for == 'STEP_2':
            init_PoI_2_Obs_NN_params_trained  = best_for_params
        
        filename_pattern = f'Inverse_{best_inv}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network/Network/{filename_pattern}')
        best_inv_params = pickle.load(open(matching_files[0], 'rb'))
        if best_inv == 'STEP_1':
            init_Obs_2_PoI_NN_params_trained, _  = best_inv_params
        if best_inv == 'STEP_2':
            init_Obs_2_PoI_NN_params_trained  = best_inv_params
        
        Obs_2_PoI_NN_accuracy_test, PoI_2_Obs_NN_accuracy_test, _, _ = acc_function((init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained), test_Observations, test_Parameters, Eigen, Sigma)
        print(f'{name_learning_approach} {num_train} {Obs_2_PoI_NN_accuracy_test * 100 : .2f} {PoI_2_Obs_NN_accuracy_test :.2e}')
        
        PoI_test_pred = PoI_2_Obs_NN(init_Obs_2_PoI_NN_params_trained, test_Observations)
        Obs_test_pred = Obs_2_PoI_NN(init_PoI_2_Obs_NN_params_trained, test_Parameters)
        
        # plot_single_true_versus_test(PoI_test_pred, test_Parameters)
        
        # if name_learning_approach == '$\TNetAEfull$':
        #     plot_single_true_versus_test(batched_Observation_operator(Obs_test_pred), test_Observations)
        # else:
        #     plot_single_true_versus_test(Obs_test_pred, test_Observations)
        full_state_predicted = from_210_to_full(Obs_test_pred)
        
        mean_max = 0.051
        std_max = 0.081
        mean_gap = 0.01
        std_gap = 0.02
        
        if (name_learning_approach == '$\mcOPOfull$') and (num_train == 1):
            mean_max = 0.41
            std_max = 0.41
            mean_gap = 0.1
            std_gap = 0.1
        
        nodal_values = (np.abs(full_state_predicted - full_true_obs)).mean(axis = 0)
        nodal_values = nodal_values[reorder_index]
        max, min, de = mean_max, 0., 0.002
        file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/{name}_trained_with{num_train}sample_mean_error_test_500sample_yobs_sample.pdf'
        plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap_plot = 'Reds', gap = mean_gap, position = 'horizontal', scale = 0.6, observed_points = True)

        nodal_values = (np.abs(full_state_predicted - full_true_obs)).std(axis = 0)
        nodal_values = nodal_values[reorder_index]
        max, min, de = std_max, -0.0, 0.002
        file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/{name}_trained_with{num_train}sample_std_error_test_500sample_yobs_sample.pdf'
        plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap_plot = 'Reds', gap = std_gap, position = 'horizontal', scale = 0.6, observed_points = True)


In [ ]:
sample = 53

In [ ]:
nodal_values = full_true_obs[sample,:]
nodal_values = nodal_values[reorder_index]

max, min, de = 3.1, 0., 0.1
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/True_test_{num_train}sample_yobs_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap_plot = 'Reds', gap = .6, position = 'horizontal', scale = 0.6, observed_points = False)

In [ ]:
nodal_values = full_state_predicted[sample,:]
nodal_values = nodal_values[reorder_index]

max, min, de = 3.1, 0., 0.1
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/Tnet_full_test_{num_train}sample_yobs_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap_plot = 'Reds', gap = .6, position = 'horizontal', scale = 0.6, observed_points = False)


In [ ]:
nodal_values = np.abs(full_true_obs[sample,:] - full_state_predicted[sample,:])
nodal_values = nodal_values[reorder_index]

max, min, de = 0.0201, 0., 0.002
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/Tnet_full_error_test_{num_train}sample_yobs_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap_plot = 'Reds', gap = .01, position = 'horizontal', scale = 0.6, observed_points = False)

In [ ]:
def from_z_to_kappa(z):
        return jnp.dot(z, jnp.transpose(jnp.dot(Eigen, Sigma)))
    
nodal_values = np.asarray(from_z_to_kappa(test_Parameters)[sample,:])
nodal_values = nodal_values[reorder_index]
max, min, de = .82, -.8, 0.1
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/True_test_sample_POI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap_plot = 'jet', gap = .4, position = 'horizontal', scale = 0.6, observed_points = False)



In [ ]:
def from_z_to_kappa(z):
        return jnp.dot(z, jnp.transpose(jnp.dot(Eigen, Sigma)))
    
nodal_values = np.asarray(from_z_to_kappa(PoI_test_pred)[sample,:])
nodal_values = nodal_values[reorder_index]
max, min, de = .82, -.8, 0.1
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/Tnet_full_test_{num_train}sample_POI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap_plot = 'jet', gap = .4, position = 'horizontal', scale = 0.6, observed_points = False)


In [ ]:
@jax.vmap
def from_z_to_kappa(z):
        return jnp.dot(z, jnp.transpose(jnp.dot(Eigen, Sigma)))
    
nodal_values = np.abs(np.asarray(from_z_to_kappa(PoI_test_pred) - from_z_to_kappa(test_Parameters))).sum(axis =1)
np.where(nodal_values == nodal_values.min())

In [ ]:
def from_z_to_kappa(z):
        return jnp.dot(z, jnp.transpose(jnp.dot(Eigen, Sigma)))
    
nodal_values = np.abs(np.asarray(from_z_to_kappa(PoI_test_pred)[sample,:] - from_z_to_kappa(test_Parameters)[sample,:]))
nodal_values = nodal_values[reorder_index]
max, min, de = .31, 0., 0.05
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/Tnet_full_test_error_{num_train}sample_POI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap_plot = 'Reds', gap = .1, position = 'horizontal', scale = 0.6, observed_points = False)


In [ ]:
def from_z_to_kappa(z):
        return jnp.dot(z, jnp.transpose(jnp.dot(Eigen, Sigma)))
    
nodal_values = np.asarray(from_z_to_kappa(Tik_POI)[sample,:])
nodal_values = nodal_values[reorder_index]
max, min, de = .82, -.8, 0.1
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/Tik_test_sample_POI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap_plot = 'jet', gap = .4, position = 'horizontal', scale = 0.6, observed_points = False)


In [ ]:
def from_z_to_kappa(z):
        return jnp.dot(z, jnp.transpose(jnp.dot(Eigen, Sigma)))
    
nodal_values = np.abs(np.asarray(from_z_to_kappa(Tik_POI)[sample,:] - from_z_to_kappa(test_Parameters)[sample,:]))
nodal_values = nodal_values[reorder_index]
max, min, de = .31, 0., 0.05
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_Poisson/Tik_test_error_sample_POI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap_plot = 'Reds', gap = .1, position = 'horizontal', scale = 0.6, observed_points = False)


## Randomness over 10 different samples for training 1 sample.

In [ ]:
Error_array = []
name_learning_approach_array = ['$\TNetAEfull$']
for name_learning_approach in name_learning_approach_array:
    for num_train in [1]:
        if (name_learning_approach == '$\TNetAEfull$'):
            name_learning = 'mc_AutoEncoder_TNet_O2P2O_modified'
            best_inv = 'STEP_1'
            
        filename_pattern = f'Inverse_Random_*'
        matching_files = glob.glob(f'Network/Network/{filename_pattern}')
            
        for matching_file in matching_files:
            best_inv_params = pickle.load(open(matching_file, 'rb'))
            init_Obs_2_PoI_NN_params_trained, _  = best_inv_params
            Obs_2_PoI_NN_accuracy_test, _, _, _ = acc_function((init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained), test_Observations, test_Parameters, Eigen, Sigma)
            Error_array.append(Obs_2_PoI_NN_accuracy_test)
            

In [ ]:
jnp.asarray(Error_array).mean(), jnp.asarray(Error_array).std()

In [ ]:
# Plot the triangulation
plt.figure(figsize=(2, 2))
# plt.triplot(triangulation, 'k-', linewidth=0.5)
plt.plot([0,0], [0,1], 'b', linewidth=2.)
plt.plot([0,1], [0,0], 'b', linewidth=2.)
plt.plot([0,1], [1,1], 'b', linewidth=2., label = r'$\Gamma^{ext}$')
plt.plot([1,1], [0,1], 'r', linewidth=2., label = r'$\Gamma^{root}$')
plt.plot(obs_coords[:,0, 0], obs_coords[:,0, 1], 'ks', markersize=3, label = 'Observed points')

# Add index next to each point in obs_coords
for i, coord in enumerate(obs_coords):
    plt.text(coord[0, 0] + 0.02, coord[0, 1]-0.01, str(i), fontsize=12)

plt.axis('equal')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
# plt.legend(ncol = 3, bbox_to_anchor=(0.5, -0.07), loc='upper center')

plt.xticks([])
plt.yticks([])

# Hide the axis lines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['left'].set_visible(False)

import tikzplotlib
tikzplotlib.save(f"../../mcAE_Overleaf/Figs/2D_Poisson/domain_observables_index.tex")

# plt.close()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

list_point = np.asarray([0, 11, 21, 31, 41, 51, 61, 71, 81, 91])  # List of points to plot

plt.figure(figsize=(6, 3))  # Increased figure size for better visibility

# Plot individual scatter points
for point in list_point:
    plt.scatter(np.arange(1, len(train_Observations.mean(axis=0))+1), train_Observations[point], c='r', alpha=0.5, s=20)

plt.scatter(np.arange(1, len(train_Observations.mean(axis=0))+1), train_Observations[point], c='r', alpha=0.5, s=20, label = 'samples')

# Calculate mean and standard deviation
x = np.arange(1, len(train_Observations.mean(axis=0))+1)  # Start x from 1
y = train_Observations.mean(axis=0)
yerr = train_Observations.std(axis=0)

# Plot errorbar
plt.errorbar(x, y, yerr=yerr, fmt='o', markersize = 3, color='b', ecolor='g', capsize=5, label='Mean with Std Dev')

plt.xlabel('Observation Index')
plt.ylabel('Observation Value')
plt.xticks([1,2,3,4,5,6,7,8,9,10])
plt.yticks([0,1,2])

plt.ylim(0, 2.5)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc = 'upper left') 
plt.tight_layout()
# plt.show()

import tikzplotlib
tikzplotlib.save(f"../../mcAE_Overleaf/Figs/2D_Poisson/observations_mean_standard_deviation.tex")

## Different noise level for 1st training sample for TNet full

In [ ]:
Error_array = []
name_learning_approach_array = ['$\TNetAEfull$']
for name_learning_approach in name_learning_approach_array:
    for num_train in [1]:
        if (name_learning_approach == '$\TNetAEfull$'):
            name_learning = 'mc_AutoEncoder_TNet_O2P2O_modified'
            best_inv = 'STEP_1'
            
        filename_pattern = f'Inverse_noise_*'
        matching_files = glob.glob(f'Network/Network/{filename_pattern}')
        noise_levels = np.sort([float(file.split('noise_')[1].split('_')[0]) for file in matching_files])
        
        # for matching_file in matching_files:
        #     print(matching_file)
        for noise_level in noise_levels:
            matching_file = f'Network/Network/Inverse_noise_{noise_level}_STEP_1mc_AutoEncoder_TNet_O2P2O_modified==data-d1_to_100_alpha_mcDNN_8000_alpha_P2O_0.0_alpha_O2P_1.0_alpha_Res_0.0'
            print(noise_level)
            best_inv_params = pickle.load(open(matching_file, 'rb'))
            init_Obs_2_PoI_NN_params_trained, _  = best_inv_params
            Obs_2_PoI_NN_accuracy_test, _, _, _ = acc_function((init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained), test_Observations, test_Parameters, Eigen, Sigma)
            Error_array.append(Obs_2_PoI_NN_accuracy_test)
            

In [ ]:
plt.figure(figsize=(3, 2))  
plt.plot(noise_levels, Error_array, 'b')
plt.scatter(noise_levels, Error_array, marker='*', color='r')
plt.ylim(0.4, 0.8), plt.xlim(-0.02, 0.42)
plt.ylabel('Relative error'), plt.xlabel('Noise level')
plt.yticks([0.4, 0.5, 0.6, 0.7, 0.8])
plt.xticks([0., 0.1, 0.2, 0.3, 0.4])
# plt.show()

import tikzplotlib
tikzplotlib.save(f"../../mcAE_Overleaf/Figs/2D_Poisson/TNet_full_noise_level_versus_relative_error.tex")